In [1]:
import os
from playsound import playsound
from dotenv import load_dotenv
import json

import pvporcupine
import openai
from langchain.tools.render import format_tool_to_openai_function

from utils.bot_tools import get_current_temperature, lights_on, search_wikipedia, scrape_news

load_dotenv()
openai.api_key = os.environ['OPENAI_API_KEY']


delimiter = "####"

AGENT_CONTEXT="""
You a use a bot which can converse or use specific tools. 
If a question is asked which uses a tool, don't assume--ask clarifying questions where needed.
When approriate ask questions at the end of responses to keep the conversation going. 
Keep answers short--only 3-4 sentences max.
"""


tools = [get_current_temperature, lights_on, search_wikipedia]
functions = [format_tool_to_openai_function(f) for f in tools]

In [ ]:
messages=[
    {"role": "system", "content": AGENT_CONTEXT},
    # {"role": "assistant", "content":""}
    # {"role": "function", "content":""}
    ]

In [ ]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0,max_tokens=2000):

  response = openai.ChatCompletion.create(
    model=model,
    temperature=temperature,
    max_tokens=max_tokens,
    messages=messages,
    functions=functions
  )
  return response.choices[0].message

In [ ]:
question = ("What time is it?")

messages.append({'role':'user', 'content':f"{delimiter} + {question} + {delimiter}"})

response = get_completion(messages)

if response.get("function_call"):
    tools = tools
    function_name = response["function_call"]["name"]
    func_to_call = next(filter(lambda x: x.name == function_name, tools), None)
    args = json.loads(response["function_call"]["arguments"])
    print(args)
    func_response = func_to_call(args)
    messages.append({"role": "function", "name": function_name, "content": func_response})
    function_return = get_completion(messages)
    print(function_return.content + "\n")

else:
    content = response.content
    messages.append({"role": "assistant", "content":f"{content}"})
    print(content + "\n")